### CAO Points Notebook

Include a Jupyter notebook called cao.ipynb that contains the following.

10% A clear and concise overview of how to load CAO points information from the
CAO website into a pandas data frame, pitched as your classmates.
***

20% A detailed comparison of CAO points in 2019, 2020, and 2021 using the functionality in pandas.
***

10% Appropriate plots and other visualisations to enhance your notebook for viewers.
***

In [1]:
# Importing required packages:
# HTTP Requests
import requests as rq

# Regular Expressions
import re

# Dates and times
import datetime as dt

# Data frames
import pandas as pd

# For Url downloads
import urllib.request as urlrq

# CAO Points for 2021 <br>
http://www.cao.ie/index.php?page=points&p=2021
***

In [2]:
# Getting data form CAO website
resp = rq.get("http://www2.cao.ie/points/l8.php")

# Confirming it is ok

resp

<Response [200]>

### Saving original dataset

In [3]:
# Getting current date and time
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime("%Y%m%d_%H%M%S")

In [4]:
# Creating a file path
path = 'data/cao2021_' + nowstr + '.html'

### Using decoding cp1252 that includes fada
***

In [5]:
# To include CAO website encoding screen

In [6]:
# Checking original server encoding
original_encoding = resp.encoding

resp.encoding = 'cp1252'
original_encoding

'iso-8859-1'

In [7]:
# Save original html file
with open(path, 'w') as f:
    f.write(resp.text)

# CAO Points for 2020 <br>
https://www.cao.ie/index.php?page=points&p=2020
http://www2.cao.ie/points/CAOPointsCharts2020.xlsx
***

# CAO Points for 2019 <br>
https://www.cao.ie/index.php?page=points&p=2019
http://www2.cao.ie/points/lvl8_19.pdf
***

Importing Camelot and reading PDF, yet only first page is read, stack overflow to the rescue https://stackoverflow.com/questions/56777241/camelot-is-reading-only-the-first-page-of-the-pdf

In [8]:
import camelot

data1 = camelot.read_pdf('http://www2.cao.ie/points/lvl8_19.pdf', pages='all')

one=data1[1].df

In [9]:
# df.export('data/cao2019_20211213_2306.csv', f='csv')

In [12]:

one=data1[0].df
one

,0,1,2,3
0,Course Code INSTITUTION and COURSE,,EOS,Mid
1,,Athlone Institute of Technology,,
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
6,AL810,Quantity Surveying,307,349
7,AL820,Mechanical and Polymer Engineering,300,358
8,AL830,General Nursing,410,429
9,AL832,Psychiatric Nursing,387,403


In [14]:
i=1

while i < 18:
    one.append(data1[i].df)
    i+=1
print('end')

end


In [15]:
one


,0,1,2,3
0,Course Code INSTITUTION and COURSE,,EOS,Mid
1,,Athlone Institute of Technology,,
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
5,AL805,Network Management and Cloud Infrastructure,329,442
6,AL810,Quantity Surveying,307,349
7,AL820,Mechanical and Polymer Engineering,300,358
8,AL830,General Nursing,410,429
9,AL832,Psychiatric Nursing,387,403


### References

In [ ]:
# Camelot https://www.youtube.com/watch?v=LoiHI-IB3lY